In [1]:
import numpy as np
import pandas as pd
import re
import gensim
import matplotlib.pyplot as plt


from konlpy.tag import Okt
from tqdm import tqdm

from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics.cluster import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_excel('./Data/실습데이터_머니케어.xlsx')

In [3]:
df.head()

,time,review
0,2023. 5. 30. 17:09,나의 가계부 지출 년 월 생활비 정산 만 원 가계부 인 가구 자취 생활비 식비 ...
1,2022. 10. 9. 22:41,신혼부부가계부 신혼부부 가계부화목 화목 신혼부부 가계부입당 화연휴 끝나고 오랜만에 ...
2,2022. 7. 2. 4:34,예금의 진실 문득 투자를 하고 경제의 흐름을 알아가니왜 예금을 했었나난 단지 ...
3,2022. 7. 6. 5:19,카카오뱅크 청년전세자금대출 연장 년 금리변동 카카오뱅크 청년전세자금대출 서류금리...
4,2022. 8. 15. 18:08,우당탕탕 다신 안가고싶은 삼성병원 가지베이컨그라탕해먹기 미친 폭우랑 전쟁하기 ...


### **데이터 전처리**

In [4]:
#특문, 띄어쓰기 제거 
def re_pattern(string):
    pattern = re.compile(r'[^a-zA-Z가-힣\s\.\?\!]')
    string = re.sub(pattern, ' ', string)

    pattern2 = re.compile(r'\s+')
    result = re.sub(pattern2, ' ', string)
    return result

In [5]:
df['re_review'] = df['review'].apply(lambda x : re_pattern(x))

In [6]:
df.shape

(1019, 3)

In [7]:
df = df[df['re_review'].apply(lambda x: len(x) > 15)]

In [8]:
okt = Okt()

In [9]:
#불용어 사전 

stopwords_df = pd.read_csv('./Data/ko-stopwords.csv') 

In [10]:
stopwords = set(stopwords_df['stopwords'])

In [11]:
#형태소 분
def okt_pos_tagging(string): #'Noun','Adjective','Verb'
    pos_words = okt.pos(string, stem= True, norm = True)
    result = [word for word, tag in pos_words if word not in stopwords if tag in {'Noun', 'Adjective', 'Verb'}]
    return result

In [12]:
tqdm.pandas() #프로세스바
df['review_words'] = df['re_review'].progress_apply(lambda x :okt_pos_tagging(x) )

100%|██████████████████████████████████████████████████████████████████████████████| 1019/1019 [03:33<00:00,  4.77it/s]


In [13]:
#벡터화 

tagged_corpus_list = []

for index, word in enumerate(df['review_words']):
    tag = 'document{}'.format(index) #id
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words = word))

### **Modeling**

In [14]:
#doc2vec 모델
model_doc2vec = Doc2Vec(vector_size= 200, 
                       alpha = 0.025,
                       min_alpha = 0.01,
                       window = 3,
                       min_count = 1,
                       dm = 1 ) # dm = 1 문맥

In [15]:
model_doc2vec.build_vocab(tagged_corpus_list)

In [16]:
model_doc2vec.train(tagged_corpus_list, total_examples=model_doc2vec.corpus_count, epochs=75)

In [17]:
#데이터 프레임에 추가

vector_list = []

for i in range(len(df)):
    doc2vec = model_doc2vec.dv['document{}'.format(i)]
    vector_list.append(doc2vec)

df['vector'] = vector_list

In [18]:
df.head()

,time,review,re_review,review_words,vector
0,2023. 5. 30. 17:09,나의 가계부 지출 년 월 생활비 정산 만 원 가계부 인 가구 자취 생활비 식비 ...,나의 가계부 지출 년 월 생활비 정산 만 원 가계부 인 가구 자취 생활비 식비 아끼...,"[가계부, 지출, 생활비, 정산, 만, 원, 가계부, 인, 가구, 자취, 생활비, ...","[-0.6520219, 0.38658994, -0.66199726, 0.033504..."
1,2022. 10. 9. 22:41,신혼부부가계부 신혼부부 가계부화목 화목 신혼부부 가계부입당 화연휴 끝나고 오랜만에 ...,신혼부부가계부 신혼부부 가계부화목 화목 신혼부부 가계부입당 화연휴 끝나고 오랜만에 ...,"[신혼부부, 가계부, 신혼부부, 가계부, 화목, 화목, 신혼부부, 가계부, 입당, ...","[-0.59568673, -1.8431644, 0.8196605, 1.8080845..."
2,2022. 7. 2. 4:34,예금의 진실 문득 투자를 하고 경제의 흐름을 알아가니왜 예금을 했었나난 단지 ...,예금의 진실 문득 투자를 하고 경제의 흐름을 알아가니왜 예금을 했었나난 단지 안정...,"[예금, 진실, 문득, 투자, 경제, 흐름, 알다, 예금, 안정, 이자, 바라보다,...","[-0.13999474, -0.515546, 0.111584924, -0.53484..."
3,2022. 7. 6. 5:19,카카오뱅크 청년전세자금대출 연장 년 금리변동 카카오뱅크 청년전세자금대출 서류금리...,카카오뱅크 청년전세자금대출 연장 년 금리변동 카카오뱅크 청년전세자금대출 서류금리승...,"[카카오, 뱅크, 청년, 전세, 자금, 대출, 연장, 금리, 변동, 카카오, 뱅크,...","[1.7493271, -1.6119434, -1.5605862, -0.7280502..."
4,2022. 8. 15. 18:08,우당탕탕 다신 안가고싶은 삼성병원 가지베이컨그라탕해먹기 미친 폭우랑 전쟁하기 ...,우당탕탕 다신 안가고싶은 삼성병원 가지베이컨그라탕해먹기 미친 폭우랑 전쟁하기 급 ...,"[탕, 다시다, 가다, 삼성, 병원, 가지, 베이컨, 그라탕, 해, 먹기, 미치다,...","[-0.19035694, -0.5377641, -0.86284584, -0.6314..."


In [19]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
#클러스터 수, 실루엣 지수 확인

n_cluster = []
clustering_score = []

X = np.stack(df['vector'].to_numpy())

X_red = TruncatedSVD(n_components=100, random_state=42).fit_transform(X)

for i in tqdm(range(2, 7)):
    model = AgglomerativeClustering(n_clusters=i, linkage='ward')
    labels = model.fit_predict(X_red)

    score = silhouette_score(
        X_red, labels,
        metric='euclidean',                 
        sample_size=min(10000, len(X_red)), 
        random_state=42
    )
    n_cluster.append(i)
    clustering_score.append(score)